In [ ]:
from azureml.core import Workspace, Experiment
from azureml.pipeline.steps import PythonScriptStep

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
vm_size = "Standard_D2_V2"
provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = 1, 
                                                                max_nodes = 4)
# create the cluster
compute_target = ComputeTarget.create(ws, "udacity-firdt-project-compute", provisioning_config)
# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it will use the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)
# For a more detailed view of current AmlCompute status, use get_status()
print(compute_target.get_status().serialize())


### YOUR CODE HERE ###




In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling( {
        "learning_rate": normal(10, 3),
        "keep_probability": uniform(0.05, 0.1),
        "batch_size": choice(16, 32, 64, 128)
    }
)


# Specify a Policy

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

    
    
# Create a SKLearn estimator for use with train.py
est = ### YOUR CODE HERE ###
import shutil
shutil.copy('./train.py',source_directory)

est = Estimator(source_directory=source_directory, 
                compute_target=compute_target, 
                entry_script='train.py', 
                conda_packages=['scikit-learn'],
               )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###


hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='validation_acc',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4)


In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
exp_hyper=Experiment(ws,"hyper_test")
model=exp_hyper.submit(hd_config,show_output=True)
best_model=model.get_best_run_by_primary_metric()

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
filename = 'source_directory/model1.sav'
joblib.dump(model, filename)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
df=TabularDatasetFactory("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    #return x_df,y_df
x, y = clean_data(df)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="AUC_weighted",
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)


In [2]:
# Submit your automl run
exp=Experiment(ws,"automl_test")
automl_model=exp.submit(automl_config,show_output=True)
### YOUR CODE HERE ###
import joblib



In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_model=automl_model.get_best_run_by_primary_metric()
joblib.dump(best_model, 'autiml_model1.sav')